# Getting an LLM summary using the API
- How would you use the API and an LLM model + prompt to generate a summary of the results obtained from the RAG pipeline?

In [1]:
from __future__ import annotations
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache
import os
import sys
# change the path to the backend directory
sys.path.append(os.path.join(os.path.dirname("."), '../../backend/'))

In [2]:
from modules.llm import get_llm_chain, get_llm_result_from_string
from modules.utils import load_config_and_device

In [3]:
# Config and DB

# load the configuration and device
config = load_config_and_device("../../backend/config.json")
config["persist_dir"] = "../../backend/data/chroma_db/"
config["data_dir"] = "../../backend/data/"

[INFO] Finding device.
[INFO] Device found: cpu


# Get LLM summary of a string
- Ensure that Ollama is running before this works ```bash ollama/.get_ollama.sh``` (or use the desktop Ollama app for testing)
- As you can tell, the data needs to be a string. To then get the results from a bunch of langchain documents, you must first concatenate the text you care about into a single string.

In [4]:
config["llm_prompt_template"] = "The following is a set of documents {docs}. Based on these docs, please summarize the content concisely. Also give a list of main concepts found in the documents. Do not add any new information. Helpful Answer: "
config["llm_model"] = "qwen2:1.5b"

In [5]:
# get the llm chain and set the cache
llm_chain = get_llm_chain(config=config, local=True)
# use os path to ensure compatibility with all operating systems
set_llm_cache(SQLiteCache(database_path=os.path.join(config["data_dir"], ".langchain.db")))

In [8]:
get_llm_result_from_string(llm_chain, "This document is about eating disorders and this one is about eating nice food")

'Eating Disorders\n\n- Eating disorders refer to psychological and emotional conditions characterized by compulsive behaviors such as overeating or excessive restriction.\n- These behaviors lead to significant weight loss, malnutrition, and serious health complications.\n\nEating Nice Food\n\n- This document focuses on the importance of eating good food for maintaining a healthy and balanced diet.\n- It highlights how selecting nutrient-dense foods can aid in overall physical and mental well-being.'